In [1]:

%%sql

--DROP DATABASE IF EXISTS SCD CASCADE;
CREATE DATABASE IF NOT EXISTS SCD;


StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 2, Finished, Available)

Error: java.lang.reflect.InvocationTargetException

In [22]:
%%sql
DROP TABLE SQLDW.SCD_DimDepartmentGroup

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 24, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [23]:
%%pyspark
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, concat
from pyspark.sql import functions as F
from pyspark.sql.functions import lit

#Reading_df = sqlContext.table("view_Columns_scd")

df = spark.read\
  .format("parquet")\
  .load("Files/Files/SCD/SCD_DimDepartmentGroup_RAW/*.parquet")

w = Window().orderBy("DepartmentGroupKey")
Reading_df_col = df.withColumn("From_date", F.current_date())\
                    .withColumn("ID_valid",lit("1")).withColumn("ID_Deleted",lit("0")).withColumn("End_date",  lit(None)).withColumn("ID_Surr", concat(row_number().over(w) + 1))
##convert the column to datetime
Reading_df_col = Reading_df_col.withColumn("End_date",col("End_date").cast("date"))
                  
Reading_df_col.show(15)

Reading_df_col.write.format("delta").mode("overwrite").saveAsTable("SQLDW.SCD_DimDepartmentGroup")

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 25, Finished, Available)

+------------------+------------------------+--------------------+----------+--------+----------+--------+-------+
|DepartmentGroupKey|ParentDepartmentGroupKey| DepartmentGroupName| From_date|ID_valid|ID_Deleted|End_date|ID_Surr|
+------------------+------------------------+--------------------+----------+--------+----------+--------+-------+
|                 1|                    null|           Corporate|2023-05-18|       1|         0|    null|      2|
|                 2|                       1|Executive General...|2023-05-18|       1|         0|    null|      3|
|                 3|                       1|Inventory Managem...|2023-05-18|       1|         0|    null|      4|
|                 4|                       1|       Manufacturing|2023-05-18|       1|         0|    null|      5|
|                 5|                       1|   Quality Assurance|2023-05-18|       1|         0|    null|      6|
|                 6|                       1|Research and Deve...|2023-05-18|   

In [11]:
from pyspark.sql import functions as F

F.current_date() 

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 13, Finished, Available)

Column<'current_date()'>

In [24]:
%%pyspark

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, concat
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.functions import max
from pyspark.sql.functions import *
from delta.tables import *


 #REad the data
df_main = spark.read\
  .format('delta')\
  .table('SQLDW.SCD_DimDepartmentGroup')


df_silver = spark.read\
  .format('delta')\
  .load("Files/Files/SCD/SCD_DimDepartmentGroup_Silver")

df_silver= df_silver.fillna(0)
df_main= df_main.fillna(0)



##got the max value
#const_surr = df_main.select(max(df_main.ID_Surr))+1

#max_id_surr = df_main.select(F.max(df_main.ID_Surr)).first()[0]
#const_surr = int(max_id_surr) + 1


###only for this POC
df_silver = df_silver.drop("surrogate_ID")
df_silver = df_silver.drop("ID_valid")
df_silver = df_silver.drop("ID_Deleted")
df_silver = df_silver.drop("Curr_date")
df_silver=df_silver.dropDuplicates(['DepartmentGroupKey','ParentDepartmentGroupKey','DepartmentGroupName'])


##Insert - Returns only columns from the left dataset for non-matched records.
df_left_forIns= df_silver.join(df_main,df_silver.DepartmentGroupKey == df_main.DepartmentGroupKey, "leftanti")



##adding the version  columns
df_left_forIns = df_left_forIns.withColumn("From_date", F.current_date())\
                   .withColumn("ID_valid",lit("1")).withColumn("ID_Deleted",lit("0")).withColumn("End_date",  lit(None))



##remove duplicated
df_left_forIns.dropDuplicates()

df_left_forIns.show()



#Everything that exist as the same in both dataframes
#first steps for the update
#using Spark SQL
df_main.createOrReplaceTempView("df_main_view")
df_silver.createOrReplaceTempView("df_silver_view")
df_left_forIns.createOrReplaceTempView("df_left_forIns_view")

df_join_forUpd=spark.sql("Select Main.DepartmentGroupKey " 
                         ",Main.ParentDepartmentGroupKey " 
                          ", Main.DepartmentGroupName " 
                          ", Main.ID_valid " 
                          ", Main.ID_Deleted " 
                          ", Main.From_date " 
                          ", Main.End_date  "
                          "From df_main_view Main Inner Join df_silver_view SILVER " 
                          "On SILVER.DepartmentGroupKey = Main.DepartmentGroupKey  "
                          "and SILVER.ParentDepartmentGroupKey = Main.ParentDepartmentGroupKey  " 
                          "and  SILVER.DepartmentGroupName = Main.DepartmentGroupName "
                          "Where not exists "
                            " (Select 1 From df_left_forIns_view Ins Where Ins.DepartmentGroupKey == Main.DepartmentGroupKey)")
df_join_forUpd.show()


##adding the version  Surrogate
#df_join_forUpd = df_join_forUpd.withColumn("ID_Surr",  F.lit(int(const_surr)))

#everyhing from the df_main that do not exist on df_join_forUpd and df_left_forIns
df_anti_forUpd = df_main.join(df_join_forUpd,df_main.DepartmentGroupKey == df_join_forUpd.DepartmentGroupKey, "leftanti")
df_anti_forUpd.show()

#df_anti_forUpd_expired = df_main.join(df_anti_forUpd,df_main.DepartmentGroupKey == df_anti_forUpd.DepartmentGroupKey, "inner").distinct(df_main.DepartmentGroupKey)
#df_anti_forUpd_expired.show()

#first make a copy
df_anti_forUpd_copy = df_anti_forUpd
#now drop the columns for the Union
df_anti_forUpd = df_anti_forUpd.drop("ID_Surr")


##Union the dataframes. Now we can merge and manage  the versions
unionDF = df_anti_forUpd.union(df_left_forIns)


# Add increasing ID column
max_id_surr = df_main.select(F.max(df_main.ID_Surr)).first()[0]
max_surr = int(max_id_surr) + 1

# Add increasing ID column
windowSpec = Window().orderBy("DepartmentGroupKey")
unionDF = unionDF.withColumn("ID_Surr", max_surr + row_number().over(windowSpec) - 1)

unionDF.show(10)
df_anti_forUpd_copy.show(10)

unionDF_complete = unionDF.union(df_anti_forUpd_copy)
unionDF_complete.show(10)


deltaTableNew = DeltaTable.forName(spark, 'SQLDW.SCD_DimDepartmentGroup')


deltaTableNew.alias('SCD_DimDepartmentGroup') \
  .merge(
    unionDF_complete.alias('updates'),
    'SCD_DimDepartmentGroup.ID_Surr = updates.ID_Surr' 
  ) \
  .whenMatchedUpdate(set =
    {
        "ID_valid": "0",
        "ID_Deleted" : "1",
        "End_date" : current_date() 
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "DepartmentGroupKey" :"updates.DepartmentGroupKey",
      "ParentDepartmentGroupKey" :"updates.ParentDepartmentGroupKey",
      "DepartmentGroupName" :"updates.DepartmentGroupName",
      "ID_Surr":"ID_Surr",
      "ID_valid" : "1",
      "ID_Deleted": "0",
      "From_date" : current_date()
     }
  ) \
  .execute()



#dfUpdates.Show()
#deltaTableNew.show()

#Read the data updated
#union with the data versioned with the new row for the update
#overwrite.

#df_main = spark.read\
 # .format('delta')\
  #.table('SCD.SCD_DimDepartmentGroup')

#df_main =  df_main.union(df_anti_forUpd)

##renamed
#df_main =  df_main.withColumnRenamed("ID_Surr", "ID_Surr_old")

##recreate column and order again
#w =  Window().orderBy("ID_Surr_old")
#df_main =df_main.withColumn("ID_Surr", concat(row_number().over(w) + 1))

##drop column
#df_main =  df_main.drop( "ID_Surr_old")


#df_main.write.format("delta").mode("overwrite").saveAsTable("SCD.SCD_DimDepartmentGroup")

df_main.show (10)

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 26, Finished, Available)

+------------------+------------------------+-------------------+----------+--------+----------+--------+
|DepartmentGroupKey|ParentDepartmentGroupKey|DepartmentGroupName| From_date|ID_valid|ID_Deleted|End_date|
+------------------+------------------------+-------------------+----------+--------+----------+--------+
|                 8|                       1|                 IT|2023-05-18|       1|         0|    null|
|                 9|                       1|           Security|2023-05-18|       1|         0|    null|
+------------------+------------------------+-------------------+----------+--------+----------+--------+

+------------------+------------------------+--------------------+--------+----------+----------+--------+
|DepartmentGroupKey|ParentDepartmentGroupKey| DepartmentGroupName|ID_valid|ID_Deleted| From_date|End_date|
+------------------+------------------------+--------------------+--------+----------+----------+--------+
|                 1|                      

In [25]:
df = spark.sql("SELECT * FROM SQLDW.scd_dimdepartmentgroup LIMIT 1000")
display(df)

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, 182285a5-abb7-4aac-9bd8-61133ebffbc2)

In [28]:
%%sql
OPTIMIZE SQLDW.SCD_DimDepartmentGroup

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 30, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

In [29]:
df = spark.sql("SELECT * FROM SQLDW.scd_dimdepartmentgroup LIMIT 1000")
display(df)

StatementMeta(, 69418c72-4691-4ae3-8f83-0599de74132a, 31, Finished, Available)

SynapseWidget(Synapse.DataFrame, b08374fe-001a-4460-9326-65e65a7844ee)